**Load the required packages and data**

In [1]:
# Load the required packages 
import numpy as np
import csv
import spacy
from collections import Counter
import math

2023-04-13 11:02:30.306648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-13 11:02:30.306734: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-13 11:02:32.011232: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-13 11:02:54.338883: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-13 11:02:54.339146: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
# Load the data 
with open("data/prototyping_sentences.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        print(row)

['sentence    structure   direct_object   indirect_object']
['The driver gave the bracelet to the grandpa', 'DO', 'bracelet', 'grandpa']
['The driver gave the grandpa the bracelet', 'PO', 'bracelet', 'grandpa']
['The veteran gave the flag to the uncle', 'DO', 'flag', 'uncle']
['The veteran gave the uncle the flag', 'PO', 'flag', 'uncle']
['The guy gave the cotton to the soldier', 'DO', 'cotton', 'soldier']
['The guy gave the soldier the cotton', 'PO', 'cotton', 'soldier']
['The astronaut gave the mustard to the boy', 'DO', 'mustard', 'boy']
['The astronaut gave the boy the mustard', 'PO', 'mustard', 'boy']
['The referee gave the clarinet to the commander', 'DO', 'clarinet', 'commander']
['The referee gave the commander the clarinet', 'PO', 'clarinet', 'commander']
['The teenager gave the pliers to the human', 'DO', 'pliers', 'human']
['The teenager gave the human the pliers', 'PO', 'pliers', 'human']
['The boy gave the loaf to the man', 'DO', 'loaf', 'man']
['The boy gave the man the l

###########################################################################################################

In [3]:
def get_sentence_structure(sentence):
    sentence_structure = None
    assert sentence_structure in {'DO', 'PO', None}
    return sentence_structure

###################################################################################################################

In [15]:
nlp = spacy.load('language_modeling_env/lib/python3.7/site-packages/en_core_web_sm/en_core_web_sm-3.5.0')
# nlp = spacy_stanza.load_pipeline("en")

def classify_sentence_structure(sentence):
    doc = nlp(sentence)
    for token in doc:
        if token.dep_ == 'dobj':
            dobj = token
            print(dobj)
        elif token.dep_ == 'dative':
            iobj = token
            print(iobj)
        elif token.dep_ == 'pobj' and token.head.pos_ == 'ADP':
            prep = token.head
            pobj = token
    if 'dobj' in locals() and 'dative' in locals():
        return 'Double Object'
    elif 'dobj' in locals() and 'pobj' in locals():
        return 'Prepositional Object'
    else:
        return 'Unknown'

In [16]:
SENTENCE = 'The bride gave the lever to the fireman'

classify_sentence_structure(SENTENCE)

lever
to


'Prepositional Object'

###################################################################################################################

###################################################################################################################

# Task 1

In [17]:
def clf_stc(sentence):
    doc = nlp(sentence)
    #Find the dependency matchers
    tokens_matchers = [matchers.dep_ for matchers in doc]
    
    if 'dative' in tokens_matchers and 'dobj' in tokens_matchers:
        # Now check if it is a PO
        if 'pobj'in tokens_matchers:
            #confirm again 
            if tokens_matchers.index("dobj") < tokens_matchers.index("dative"):
                sentence_structure = "PO"
        else:
            # Check for possible DO
            tokens_matchers.index("dative") < tokens_matchers.index("dobj")
            sentence_structure = "DO"
            
    else:
        sentence_structure =  None
        
    return sentence_structure

In [18]:
# testing 
SENTENCE = "[laughter] my daughter handed me the phone and said here dad wake up"


clf_stc(SENTENCE)

'DO'

In [19]:
# import spacy_stanza

# nlp = spacy_stanza.load_pipeline("en")

In [24]:
from spacy.matcher import DependencyMatcher


# Define the patterns for PO and DO
patterns = [[
    # Pattern for prepositional object (PO)
    {
        "RIGHT_ID": "prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
        "LEFT_ID": "obj",
        "LEFT_ATTRS": {"DEP": {"IN": ["dobj", "dative"]}},
    },
    # Pattern for double object (DO)
    {
        "LEFT_ID": "dobj",
        "LEFT_ATTRS": {},
        "RIGHT_ID": "dative",
        "RIGHT_ATTRS": {"DEP": "dative", "ENT_TYPE": {"NOT_IN": ["GPE", "LOC"]}},
    },
]]

matcher = DependencyMatcher(nlp.vocab)
matcher.add("PO_DO", patterns)

def extract_indirect_object(sentence):
    doc = nlp(sentence)
    matches = matcher(doc)
    if matches and matches[0][0] == 1:
        dobj = matches[0][1]
        iobj = next((token for token in dobj.children if token.dep_ == "dative"), None)
        if iobj:
            return iobj.text
    return None

ValueError: [E099] Invalid pattern: the first node of pattern should be an anchor node. The node should only contain RIGHT_ID and RIGHT_ATTRS.

In [9]:
# Extract direct objects 
def extract_direct_object(sentence):
    # split the sentence into list
    tokens = sentence.split()
    direct_object = ""
    
    doc = nlp(sentence)
    tokens_matchers = [matchers.dep_ for matchers in doc]
    
    for i in range(len(tokens)):
        print(tokens[i], ",", tokens_matchers[i])
    
    # get the position of the direct object
    dobj_index = tokens_matchers.index("dobj")
    # now find the object
    direct_object_word = tokens[dobj_index]
    
    object_phrases = [chunk.text for chunk in doc.noun_chunks]
    # Get the full direct object 
    for chunk in object_phrases:
        if direct_object_word in chunk:
            direct_object = chunk
        else:
            direct_object = direct_object_word
    return direct_object

In [10]:
extract_direct_object(SENTENCE)

[laughter] , dep
my , dep
daughter , dep
handed , poss
me , nsubj
the , ROOT
phone , dative
and , det
said , dobj
here , cc
dad , conj
wake , advmod
up , nsubj


'said'

In [11]:
######################################################################################################

In [12]:
# Extract indirect objects 
def extract_indirect_object(sentence):
    doc = nlp(sentence)
    tokens_matchers = [matchers.dep_ for matchers in doc]
    
    # get the sentence structure
    sentence_structure = clf_stc(sentence)
    
    indirect_object_word = ""
    
    if sentence_structure == 'DO':
        indobj_index = tokens_matchers.index('dative')
        indirect_object_word = doc[indobj_index]
    elif sentence_structure == 'PO':
        indobj_index = tokens_matchers.index('pobj')
        indirect_object_word = doc[indobj_index]
    else:
        pass
    
    object_phrases = [chunk.text for chunk in doc.noun_chunks]
    #Get the indirect objects
    for chunk in object_phrases:
        if str(indirect_object_word) in chunk:
            indirect_object = chunk

    return indirect_object

In [13]:
extract_indirect_object(SENTENCE)

'me'

In [14]:
stop here

SyntaxError: invalid syntax (4067800170.py, line 1)

# Task 2

In [ ]:
import nltk

In [ ]:
def extract_feature_1(noun_phrase, sentence):
    return len(nltk.word_tokenize(noun_phrase))



In [ ]:
SENTENCE = "The bride gave the fireman the lever"

extract_feature_1("fireman", SENTENCE)

In [ ]:
def extract_feature_2(noun_phrase, sentence):
    pos_tags = [pos for (word, pos) in nltk.pos_tag(nltk.word_tokenize(noun_phrase))]
    return ' '.join(pos_tags)



In [ ]:
extract_feature_2("the fireman", SENTENCE)

In [ ]:
def extract_feature_3(noun_phrase, sentence):
    words = nltk.word_tokenize(noun_phrase)
    word_freqs = Counter(words)
    total_words = len(words)
    log_mean_freq = math.log(sum(word_freqs.values()) / total_words)
    log_median_freq = math.log(sorted(word_freqs.values())[total_words // 2])
    return log_mean_freq



In [ ]:
extract_feature_3("fireman", SENTENCE)

In [ ]:
def concatenate_features(features1, features2):
    features = [features1, features2]
    #for i in range(len(features1)):
        #features.append(features1[i] + features2[i])
    return features

In [ ]:
features1 = extract_feature_1("the fireman", SENTENCE)
features2 = extract_feature_2("the fireman", SENTENCE)

concatenate_features(features1, features2)

# Task 3

In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Define a list of sentences
sentences = ['This is an example sentence', 'Each sentence is converted to an embedding']

# Compute embeddings for all sentences
embeddings = model.encode(sentences)

# Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)

In [ ]:
import nltk
from nltk.corpus import wordnet
import random

def alter_sentence(sentence):
    # Tokenize the sentence
    tokens = nltk.word_tokenize(sentence)
    
    # Get the part of speech of each word in the sentence
    pos_tags = nltk.pos_tag(tokens)
    
    # Choose a random word to replace
    word_to_replace = random.choice(pos_tags)[0]
    
    # Get the synset of the word
    synset = wordnet.synsets(word_to_replace)
    
    # Choose a random synset
    random_synset = random.choice(synset)
    
    # Get a random word from the chosen synset
    random_word = random_synset.lemmas()[random.randint(0, len(random_synset.lemmas())-1)].name()
    
    # Replace the chosen word with the random word
    altered_sentence = ' '.join([random_word if word == word_to_replace else word for word in tokens])
    
    return altered_sentence

In [ ]:
# Define the original sentence
sentence = "The quick brown fox jumps over the lazy guy."

# Produce an altered version of the sentence
altered_sentence = alter_sentence(sentence)

# Print the original and altered sentences
print("Original sentence:", sentence)
print("Altered sentence:", altered_sentence)